In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from scripts.gamescreen.scripts import *

In [ ]:
car_path = os.path.abspath(os.path.join(os.path.dirname("__file__"), 'users'))
file_path = os.path.join(car_path, 'registro_partidas.csv')
datos = pd.read_csv(file_path)

In [ ]:
def level_graphic():
    """ Porcentaje de partidas por nivel. """
    
    plt.clf()

    facil = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Facil')]['nivel'].count()
    normal = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Normal')]['nivel'].count()
    dificil = datos[(datos['estado'] == 'finalizada') & (datos['nivel'] == 'Dificil')]['nivel'].count()

    data_dibujo = [facil, normal, dificil]
    etiquetas = ['Facil', 'Normal', 'Dificil']
    plt.pie(data_dibujo, labels=etiquetas, autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
    plt.legend(etiquetas)
    plt.title('Porcentaje de partidas finalizadas por dificultad')
    plt.show()

In [ ]:
def partidas_estado():
    """Gráfico que muestre el porcentaje de partidas por estado (fin, cancelada, error, timeout).
        También debe tener en cuenta las partidas que se terminaron inesperadamente o de forma repentina
        (partidas sin evento de fin)"""

    datos = pd.read_csv(file_path)

    fin = datos[(datos['estado']) == 'finalizada']['estado'].count()
    #cierre por falla -> error
    col_estado = datos.loc['estado']
    cant_error =

    error = datos[(datos['estado']) == 'error']['estado'].count()
    timeout = datos[(datos['estado']) == 'timeout']['estado'].count()
    cancelada = datos[(datos['estado']) == 'cancelada']['estado'].count()

    data_graf = [fin, error, timeout, cancelada]
    etiquetas = ['fin', 'error', 'timeout', 'cancelada']
    plt.pie(data_graf, labels=etiquetas, autopct='%1.1f%%', shadow=True, startangle=90, labeldistance=1.1)
    plt.legend(etiquetas)
    plt.title('Porcentaje de partidas por estado')
    plt.show()
